In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy import signal

def arnold_cat_map(image, iterations=1, forward=True):
    """Aplica el Arnold Cat Map (igual que en tu versión original)"""
    N = image.shape[0]
    transformed = np.copy(image)
    
    for _ in range(iterations):
        temp = np.zeros_like(transformed)
        for x in range(N):
            for y in range(N):
                if forward:
                    new_x = (2*x + y) % N
                    new_y = (x + y) % N
                else:
                    new_x = (x - y) % N
                    new_y = (-x + 2*y) % N
                temp[new_x, new_y] = transformed[x, y]
        transformed = temp
    return transformed

def find_period(image):
    """Encuentra el período (igual que en tu versión original)"""
    N = image.shape[0]
    transformed = np.copy(image)
    period = 0
    
    while True:
        period += 1
        transformed = arnold_cat_map(transformed, 1, forward=True)
        
        if np.array_equal(transformed, image):
            return period
        
        if period > 3*N:
            return -1

def calculate_correlation(img1, img2):
    """Calcula correlación (igual que en tu versión original)"""
    return signal.correlate2d(img1, img2, mode='same', boundary='fill', fillvalue=0).max()

def main():
    ######################################################
    ### PARTE MODIFICADA: Carga de la imagen personalizada
    ######################################################
    try:
        # Reemplaza "tu_imagen.jpg" con la ruta de TU archivo
        image_path = "fresa.jpeg"  # ← Cambia esto
        
        # Carga y preprocesamiento de la imagen
        original_img = np.array(
            Image.open(image_path)
            .convert('L')          # Convierte a escala de grises
            .resize((255, 255))    # Ajusta al tamaño requerido
        print(f"✔ Imagen cargada: {image_path}")
        
    except FileNotFoundError:
        print("⚠ No se encontró la imagen. Generando patrón de prueba...")
        original_img = np.zeros((255, 255), dtype=np.uint8)
        for i in range(255):
            original_img[i, :] = i
    ######################################################

    # Resto del código original (sin cambios)
    transformed_img = arnold_cat_map(original_img, iterations=1)
    period = find_period(original_img)
    print(f"El período es: {period} iteraciones")
    
    recovered_img = arnold_cat_map(transformed_img, iterations=period-1)
    
    min_corr_img = None
    min_corr = float('inf')
    for i in range(1, period):
        temp_img = arnold_cat_map(original_img, iterations=i)
        corr = calculate_correlation(original_img, temp_img)
        if corr < min_corr:
            min_corr = corr
            min_corr_img = temp_img
    
    # Visualización
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(original_img, cmap='gray')
    plt.title('Original')
    
    plt.subplot(1, 3, 2)
    plt.imshow(recovered_img, cmap='gray')
    plt.title('Recuperada')
    
    plt.subplot(1, 3, 3)
    plt.imshow(min_corr_img, cmap='gray')
    plt.title(f'Menor correlación\n(corr={min_corr:.2f})')
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()